# Contexto
- Spotify ya no provee de acceso a ciertos api Points desde Noviembre del 2024
- Encontré en https://gist.github.com/andytlr/4104c667a62d8145aa3a el siguiente comentario:
    - Oh, I found a better list really, I found this chosic site that splits it into genres and subgenres and I extracted it into a json file that I'll attach here (https://drive.google.com/file/d/1w7H_myVKMI1XIx2rPQb9hlr8ZoXXz3W8/view?usp=drivesdk). Use the "genres" key to get the main genres and the "subgenres" key to get all subgenres and the "genre_map" to get the dict that has each main genres as keys and their subgenres as values.

    - The site is https://www.chosic.com/list-of-music-genres/, it has 5600+ total subgenres, I found another one that has 6200 genres tho but no subcategory breakdown. (https://everynoise.com/everynoise1d.html)

    - Also the offset doesn't let you get past the 1000 limit, it first has a range limit so the highest value of offset you can use is 1000 but if you are using Limit is 50, it's 950.

- He descargado el .json y voy a mapear los subgéneros a géneros de forma que podamos obtener más información!
- Durante el progreso he ido engordando el JSON con los géneros que no tenía están mapeados de forma que el json cada vez nos de más información

In [1]:
import sys
sys.path.append("../")
import src.soporte_sql as sql
import pandas as pd
import ast
import json

# Tabla main_genres
- Tiene el nombre del género
- Tiene las veces que aparece (para el ranking)
- Conectado a la marca del que proviene

In [ ]:
def mapeo_genres():
    with open("../datos/00_Spotify_Genres/genres_dict.json", "r") as file:
        genres_mapping = json.load(file)
    
    dictio_genres = {}
    for main_genre, subgenres in genres_mapping["genres_map"].items():
        for subgenre in subgenres:
            dictio_genres[subgenre] = main_genre
    return dictio_genres

In [ ]:
def obtener_main_genres(generos_df,brand_id = 0, ruta_csv = "../datos/02 Base de Datos/tempsave_check_genre.csv"):
    conexion = sql.conectar_bd()
    query = ''' SELECT * FROM brands'''
    df = sql.consulta_sql(conexion, query)
    df = df[["id","name"]]
    check = df["id"].tolist()
    if brand_id not in check:
        print("El id de la marca no existe")
        print("Los ids de las marcas son:")
        return display(df)
    
    generos = generos_df[["brand","genres_ranking"]]
    generos["genres_ranking"] = generos["genres_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
    
    lista_brands = []
    lista_generos = []
    lista_conteos = []
    for tuplas in generos["genres_ranking"]:
        for artista, conteo in tuplas:
            lista_generos.append(artista)
            lista_conteos.append(conteo)
            lista_brands.append(brand_id) # 1 es zara en la base de datos

    generos = pd.DataFrame({
        "subgenre_name" : lista_generos,
        "number_of_appearances" : lista_conteos,
        "brand_id" : lista_brands
    })

    # LLamamos al JSON y generamos el diccionario de Mapeo de subgeneros a generos principales
    mapping_dict = mapeo_genres()
    # Mapeamos los subgeneros a los generos principales
    generos["genre"] = generos["subgenre_name"].map(mapping_dict)
    return generos

def subir_main_genres(generos_df,brand_id = 0, ruta_csv = "../datos/02 Base de Datos/tempsave_main_genre.csv"):
    # Obtenemos el conteo de apariciones por genero principal
    main_generos = main_generos.groupby("genre")["number_of_appearances"].sum().reset_index()
    # Ordenamos por numero de apariciones
    main_generos.sort_values(by="number_of_appearances",ascending=False,inplace=True)
    main_generos.reset_index(drop=True,inplace=True)
    main_generos["brand_id"] = brand_id
    # Guardar en csv
    main_generos.to_csv(ruta_csv,index=False)
    conexion = sql.conectar_bd()
    query = '''INSERT INTO main_genres(genre_name,number_of_appearances,brand_id) VALUES (%s,%s,%s)'''
    sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(main_generos))

In [2]:
generos = pd.read_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv")
generos = generos[["brand","genres_ranking"]]
generos["genres_ranking"] = generos["genres_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [3]:
lista_brands = []
lista_generos = []
lista_conteos = []
for tuplas in generos["genres_ranking"]:
    for artista, conteo in tuplas:
        lista_generos.append(artista)
        lista_conteos.append(conteo)
        lista_brands.append(1) # 1 es zara en la base de datos

generos = pd.DataFrame({
    "subgenre_name" : lista_generos,
    "number_of_appearances" : lista_conteos,
    "brand_id" : lista_brands
})
generos.to_csv("../datos/02 Base de Datos/07_ranking_y_generos_zara.csv",index=False)
generos.head()

,subgenre_name,number_of_appearances,brand_id
0,lo-fi,1637,1
1,lo-fi beats,1235,1
2,afro house,1195,1
3,tech house,1112,1
4,classical,1075,1


### Usando el json, mapeamos los subgeneros a los géneros

In [4]:
with open("../datos/00_Spotify_Genres/genres_dict.json", "r") as file:
    genres_mapping = json.load(file)

In [5]:
dictio_genres = {}
for main_genre, subgenres in genres_mapping["genres_map"].items():
    for subgenre in subgenres:
        dictio_genres[subgenre] = main_genre

In [6]:
generos["genre"] = generos["subgenre_name"].map(dictio_genres)

In [50]:
main_generos = generos.copy()
subgeneros = generos.copy()

### En main_generos, nos quedamos con los generos principales y cuantas veces aparecen

In [51]:
main_generos = main_generos.groupby("genre")["number_of_appearances"].sum().reset_index()
main_generos.sort_values(by="number_of_appearances",ascending=False,inplace=True)
main_generos.reset_index(drop=True,inplace=True)
main_generos["brand_id"] = 1
main_generos.to_csv("../datos/02 Base de Datos/07_main_generos_zara.csv",index=False)
main_generos.head()

,genre,number_of_appearances,brand_id
0,Electronic,28853,1
1,Pop,18628,1
2,Rock,11806,1
3,Latin,10755,1
4,Hip Hop,10357,1


### Crear tabla y subir datos

In [33]:
conexion = sql.conectar_bd()
query = ''' create table main_genres (
            id bigint primary key generated always as identity,
            genre_name text not null,
            number_of_appearances int not null,
            brand_id bigint references brands (id) on delete cascade
          );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [35]:
conexion = sql.conectar_bd()
query = '''INSERT INTO main_genres(genre_name,number_of_appearances,brand_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(main_generos))

Conectado a la base de datos
Se han añadido los valores correctamente


# Tabla subgenres
- Tiene el nombre del subgénero
- Tiene las veces que aparece (para el ranking)
- Conectado del main_genre al que pertenece
- Conectado del brand al que pertenece


In [52]:
subgeneros = subgeneros[["subgenre_name","number_of_appearances","genre","brand_id"]]
subgeneros.rename(columns={"genre":"genre_name"},inplace=True)
subgeneros.head()

,subgenre_name,number_of_appearances,genre_name,brand_id
0,lo-fi,1637,Rock,1
1,lo-fi beats,1235,Electronic,1
2,afro house,1195,Electronic,1
3,tech house,1112,Electronic,1
4,classical,1075,Classical,1


### LLamamos a la tabla de géneros principales
- Para mapear el genre al id de la base de datos

In [67]:
brand_id = 1
conexion = sql.conectar_bd()
query = f''' SELECT * FROM main_genres WHERE brand_id = {brand_id}'''
sql.consulta_sql(conexion, query)

Conectado a la base de datos


,id,genre_name,number_of_appearances,brand_id
0,1,Electronic,28853,1
1,2,Pop,18628,1
2,3,Rock,11806,1
3,4,Latin,10755,1
4,5,Hip Hop,10357,1
5,6,World/Traditional,10093,1
6,7,R&B,6849,1
7,8,Jazz,5641,1
8,9,Metal,3528,1
9,10,Classical,3481,1


In [53]:
conexion = sql.conectar_bd()
query = ''' SELECT * FROM main_genres '''
main_genres_bd = sql.consulta_sql(conexion, query)
main_genres_bd = main_genres_bd[["genre_name","id"]]
main_genres_bd

Conectado a la base de datos


,genre_name,id
0,Electronic,1
1,Pop,2
2,Rock,3
3,Latin,4
4,Hip Hop,5
5,World/Traditional,6
6,R&B,7
7,Jazz,8
8,Metal,9
9,Classical,10


In [54]:
subgeneros = subgeneros.merge(main_genres_bd,how="left",on="genre_name")
subgeneros = subgeneros[["subgenre_name","number_of_appearances","id","brand_id"]]
subgeneros.rename(columns={"id":"main_genre"},inplace=True)
subgeneros

,subgenre_name,number_of_appearances,main_genre,brand_id
0,lo-fi,1637,3,1
1,lo-fi beats,1235,1,1
2,afro house,1195,1,1
3,tech house,1112,1,1
4,classical,1075,10,1
...,...,...,...,...
658,dance pop,4,2,1
659,dance,3,1,1
660,punto guajiro,1,6,1
661,pop rap,1,5,1


### Crear tabla y subir datos

In [55]:
conexion = sql.conectar_bd()
query = ''' create table subgenres (
            id bigint primary key generated always as identity,
            subgenre_name text not null,
            number_of_appearances int not null,
            main_genre bigint references main_genres (id) on delete cascade,
            brand_id bigint references brands (id) on delete cascade
          );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [57]:
conexion = sql.conectar_bd()
query = '''INSERT INTO subgenres(subgenre_name,number_of_appearances,main_genre,brand_id) VALUES (%s,%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(subgeneros))

Conectado a la base de datos
Se han añadido los valores correctamente


In [69]:
subgeneros.to_csv("../datos/02 Base de Datos/08_subgeneros_zara.csv",index=False)